<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

***Some parts of the notebook are almost the copy of [ mmta-team course](https://github.com/mmta-team/mmta_fall_2020). Special thanks to mmta-team for making them publicly available. [Original notebook](https://github.com/mmta-team/mmta_fall_2020/blob/master/tasks/01_word_embeddings/task_word_embeddings.ipynb).***

<b> Прочитайте семинар, пожалуйста, для успешного выполнения домашнего задания. В конце ноутка напишите свой вывод. Работа без вывода оценивается ниже.

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления 

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [1]:
!wget https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1

--2021-09-27 07:57:19--  https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1453905423 (1.4G) [application/octet-stream]
Saving to: ‘SO_vectors_200.bin?download=1’

SO_vectors_200.bin? 100%[===================>]   1.35G  7.67MB/s    in 2m 41s  

2021-09-27 08:00:02 (8.61 MB/s) - ‘SO_vectors_200.bin?download=1’ saved [1453905423/1453905423]



In [2]:
from gensim.models.keyedvectors import KeyedVectors
wv_embeddings = KeyedVectors.load_word2vec_format("SO_vectors_200.bin?download=1", binary=True)

#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [3]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

float32 (200,)


In [4]:
print(f"Num of words: {len(wv_embeddings.index2word)}")

Num of words: 1787145


Найдем наиболее близкие слова к слову `dog`:

#### Вопрос 1:
* Входит ли слово `cat` в топ-5 близких слов к слову `dog`? Какое место? 

In [5]:
# method most_simmilar

wv_embeddings.most_similar(word)[:5]

[('animal', 0.8564180135726929),
 ('dogs', 0.7880867123603821),
 ('mammal', 0.7623804807662964),
 ('cats', 0.7621253728866577),
 ('animals', 0.760793924331665)]

Ответ: слово `cat` не входит в топ-5, но входит слово `cats` на 4-м месте

### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [51]:
import numpy as np
import re
import nltk
wnl = nltk.WordNetLemmatizer()

# you can use your tokenizer
# for example, from nltk.tokenize import WordPunctTokenizer

class MyTokenizer_re:
    def __init__(self):
        pass
    def tokenize(self, text):
        tokens = re.findall('\w+', text) # буквы, цифры
        tokens = [word.lower() for word in tokens]
        return tokens 

class MyTokenizer_re_lem:
    def __init__(self):
        pass
    def tokenize(self, text):
        tokens = re.findall('\w+', text)
        tokens = [word.lower() for word in tokens]
        tokens = [wnl.lemmatize(word) for word in tokens] # лемматизация
        return tokens 

In [52]:
from nltk.tokenize import WordPunctTokenizer
wordpunct_tokenize = WordPunctTokenizer().tokenize

class MyTokenizer_W:
    def __init__(self):
        pass
    def tokenize(self, text):
      tokens = wordpunct_tokenize(text) # буквы, цифры, символы
      tokens = [word.lower() for word in tokens]
      return tokens

class MyTokenizer_W_lem:
    def __init__(self):
        pass
    def tokenize(self, text):
      tokens = wordpunct_tokenize(text)
      tokens = [word.lower() for word in tokens]
      tokens = [wnl.lemmatize(word) for word in tokens] # лемматизация
      return tokens

In [62]:
tokenizer_re = MyTokenizer_re()
tokenizer_re_lem = MyTokenizer_re_lem()
tokenizer_W = MyTokenizer_W()
tokenizer_W_lem = MyTokenizer_W_lem()

tokenizers_list = [tokenizer_re, tokenizer_re_lem, tokenizer_W, tokenizer_W_lem]

In [63]:
s = "Good muffins cost $3.88 in New York. Please buy me two of them. Thanks."

for tokenizer in tokenizers_list:
  print(tokenizer.tokenize(s))

['good', 'muffins', 'cost', '3', '88', 'in', 'new', 'york', 'please', 'buy', 'me', 'two', 'of', 'them', 'thanks']
['good', 'muffin', 'cost', '3', '88', 'in', 'new', 'york', 'please', 'buy', 'me', 'two', 'of', 'them', 'thanks']
['good', 'muffins', 'cost', '$', '3', '.', '88', 'in', 'new', 'york', '.', 'please', 'buy', 'me', 'two', 'of', 'them', '.', 'thanks', '.']
['good', 'muffin', 'cost', '$', '3', '.', '88', 'in', 'new', 'york', '.', 'please', 'buy', 'me', 'two', 'of', 'them', '.', 'thanks', '.']


In [7]:
def question_to_vec(question, embeddings, tokenizer, dim=200):
    """
        question: строка
        embeddings: наше векторное представление
        dim: размер любого вектора в нашем представлении
        
        return: векторное представление для вопроса
    """

    features = np.zeros(dim, dtype='float32')
    word_count = 0

    for word in tokenizer.tokenize(question):
      try:
        features += embeddings.get_vector(f'{word}')
        word_count += 1
      except:
        pass  

    if word_count == 0:
         return features
    
    return features / word_count

Теперь у нас есть метод для создания векторного представления любого предложения.

#### Вопрос 2:
* Какая третья(с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

In [58]:
sent = 'I love neural networks'

for tokenizer in tokenizers_list:
  vec = question_to_vec(sent, wv_embeddings, tokenizer, dim=200)
  print(round(vec[2], 2))

# -1.29 без лемматизации
# -1.41 с лемматизацией 

-1.29
-1.41
-1.29
-1.41


### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями. 

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv 
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа

#### Вопрос 3:
* Максимум `Hits@47 - DCG@1`?

Ответ: Максимум Hits@47 - DCG@1 достигается при максимуме Hits@47 и минимуме DCG@1. 
Максимум функции Hits@47 - это 1, а минимум DCG@1 - это 0, тогда ответ - это 1.

<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера. 
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1)] = 0$
- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### Вопрос 4:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)

Ответ: [K = 10] $\text{DCG@10} = \frac{1}{\log_2(1+9)}\cdot[9 \le 10] = \frac{1}{\log_2{10}} = 0.3$

### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$. $dup\_ranks$ является списком, который содержит рейтинги дубликатов(их позиции в ранжированном списке). Например, $dup\_ranks = [2]$ для примера, описанного выше.

In [9]:
def hits_count(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть  Hits@k
    """

    dup_ranks = np.array(dup_ranks)
    mask = dup_ranks <= k
    hits_value = np.mean(mask)

    return hits_value    

In [10]:
def dcg_score(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть DCG@k
    """

    dup_ranks = np.array(dup_ranks)
    mask = dup_ranks <= k
    mask = mask / np.log2(1 + dup_ranks)
    dcg_value = np.mean(mask)

    return dcg_value

Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [11]:
import pandas as pd

In [12]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наши кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]
# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dup_ranks = [2]

# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

Ваш ответ HIT: [0.0, 1.0, 1.0, 1.0]
Ваш ответ DCG: [0.0, 0.63093, 0.63093, 0.63093]


У вас должно получиться

In [13]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], 
                                [0, 
                                 1 / (np.log2(3)), 
                                 1 / (np.log2(3)), 
                                 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [14]:
!pip install gdown
!gdown https://drive.google.com/uc?id=1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_

Downloading...
From: https://drive.google.com/uc?id=1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_
To: /content/stackoverflow_similar_questions.zip
131MB [00:00, 155MB/s]


In [15]:
!unzip stackoverflow_similar_questions.zip

Archive:  stackoverflow_similar_questions.zip
   creating: data/
  inflating: data/.DS_Store          
   creating: __MACOSX/
   creating: __MACOSX/data/
  inflating: __MACOSX/data/._.DS_Store  
  inflating: data/train.tsv          
  inflating: data/validation.tsv     


Считайте данные.

In [16]:
def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
        data.append(line.split('\t'))
    return data

Нам понадобиться только файл validation.

In [17]:
validation_data = read_corpus('./data/validation.tsv')

Кол-во строк

In [18]:
len(validation_data)

3760

Размер нескольких первых строк

In [21]:
for i in range(5):
    print(i + 1, len(validation_data[i]) )

1 1001
2 1001
3 1001
4 1001
5 1001


### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [22]:
from sklearn.metrics.pairwise import cosine_similarity
from copy import deepcopy

In [23]:
def rank_candidates(question, candidates, embeddings, tokenizer, dim=200):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """
    
    question_ = deepcopy(question)
    candidates_ = deepcopy(candidates)
    number_of_candidates = len(candidates_)

    # вектор для question_
    question_V = question_to_vec(question_, embeddings, tokenizer, dim) 
    question_V = question_V.reshape(1, -1)

    # список векторов для candidates_
    candidates_V = [question_to_vec(sent, embeddings, tokenizer, dim) \
                    for sent in candidates_] 
    
    # переводим список в np.array
    candidates_V = np.array(candidates_V)
    
    # считаем косинусную близость
    similarity = cosine_similarity(question_V, candidates_V).tolist()[0]

    sim_list = []

    for idx, candidate in enumerate(candidates_):

      # соединяем кандидатов с их косинусной близостью к вопросу
      sim_list.append((similarity[idx], idx, candidate))
        
    # сортируем список кандидатов по similarity
    ss_list = sorted(sim_list, key=lambda sim_list: sim_list[0], reverse=True)   
    
    # убираем similarity, 
    # оставляем в списке только пары (позиция в исходном списке кандидатов, кандидат)
    ss_list = list(map(lambda x:x[1:], ss_list)) 
    return ss_list

Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [24]:
questions = ['converting string to list', 'Sending array via Ajax fails'] 

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],
              
              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [67]:
tokenizers_dict = {'re': tokenizer_re, 're_lem': tokenizer_re_lem, 
                   'W': tokenizer_W, 'W_lem': tokenizer_W_lem}

In [68]:
for name, tokenizer in tokenizers_dict.items():
  print(name)
  for question, q_candidates in zip(questions, candidates):
    ranks = rank_candidates(question, q_candidates, wv_embeddings, tokenizer)
    print(ranks)
    print()

# без лемматизации - 102 021
# с лемматизацией - 012 021

re
[(1, 'C# create cookie from string and send it'), (0, 'Convert Google results object (pure js) to Python object'), (2, 'How to use jQuery AJAX for an outside domain?')]

[(0, 'Getting all list items of an unordered list in PHP'), (2, 'select2 not displaying search results'), (1, 'WPF- How to update the changes in list item of a list')]

re_lem
[(0, 'Convert Google results object (pure js) to Python object'), (1, 'C# create cookie from string and send it'), (2, 'How to use jQuery AJAX for an outside domain?')]

[(0, 'Getting all list items of an unordered list in PHP'), (2, 'select2 not displaying search results'), (1, 'WPF- How to update the changes in list item of a list')]

W
[(1, 'C# create cookie from string and send it'), (0, 'Convert Google results object (pure js) to Python object'), (2, 'How to use jQuery AJAX for an outside domain?')]

[(0, 'Getting all list items of an unordered list in PHP'), (2, 'select2 not displaying search results'), (1, 'WPF- How to update the change

Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента ответы на кандидаты будут <b>скрыты</b>(*)

In [71]:
# должно вывести
results = [[(1, 'C# create cookie from string and send it'),
            (0, 'Convert Google results object (pure js) to Python object'),
            (2, 'How to use jQuery AJAX for an outside domain?')],
           [(*, 'Getting all list items of an unordered list in PHP'), #скрыт
            (*, 'select2 not displaying search results'), #скрыт
            (*, 'WPF- How to update the changes in list item of a list')]] #скрыт

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

#### Вопрос 5:
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?

Ответ: `021`

Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

In [26]:
from tqdm.notebook import tqdm

In [77]:
for name, tokenizer in tokenizers_dict.items():
  print(name)
  print()

  wv_ranking = []
  max_validation_examples = 1000
  for i, line in enumerate(tqdm(validation_data)):
      if i == max_validation_examples:
          break
      q, *ex = line
      ranks = rank_candidates(q, ex, wv_embeddings, tokenizer)
      
      wv_ranking.append([r[0] for r in ranks][0]+1) 

  # среди всех дубликатов (r[0] for r in ranks) выбираем номер самого лучшего дубликата, 
  # т.е. стоящего на первом месте ([0]) и прибавляем к нему единицу, 
  # т.к. для подсчета метрик ранги должны начинаться с 1, а не с 0

  for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % \
          (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))
  print()    

re



  0%|          | 0/3760 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.415 | Hits@   1: 0.415
DCG@   5: 0.416 | Hits@   5: 0.418
DCG@  10: 0.418 | Hits@  10: 0.422
DCG@ 100: 0.426 | Hits@ 100: 0.465
DCG@ 500: 0.457 | Hits@ 500: 0.716
DCG@1000: 0.487 | Hits@1000: 1.000

re_lem



  0%|          | 0/3760 [00:00<?, ?it/s]

...

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.410 | Hits@   1: 0.410
DCG@   5: 0.411 | Hits@   5: 0.412
DCG@  10: 0.412 | Hits@  10: 0.416
DCG@ 100: 0.421 | Hits@ 100: 0.462
DCG@ 500: 0.453 | Hits@ 500: 0.715
DCG@1000: 0.483 | Hits@1000: 1.000

W_lem



  0%|          | 0/3760 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.399 | Hits@   1: 0.399
DCG@   5: 0.399 | Hits@   5: 0.400
DCG@  10: 0.401 | Hits@  10: 0.405
DCG@ 100: 0.409 | Hits@ 100: 0.449
DCG@ 500: 0.442 | Hits@ 500: 0.712
DCG@1000: 0.472 | Hits@1000: 1.000



### Эмбеддинги, обученные на корпусе похожих вопросов

In [32]:
train_data = read_corpus('./data/train.tsv')

In [34]:
len(train_data)

1000000

In [35]:
# максимум сколько вопросов в одном объекте выборки 
max(map(len, train_data))

95

Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

In [36]:
words = [tokenizer.tokenize(' '.join(x)) for x in train_data] # MyTokenizer_W с лемматизацией

In [37]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(words, # data for model to train on
                 size=200,                 # embedding vector size
                 min_count=5,             # consider words that occured at least 5 times
                 window=6).wv  # При выборе из ширины окна [3, 4, 5, 6] при window=6 был лучший результат

In [76]:
for name, tokenizer in tokenizers_dict.items():
  print(name)
  print()

  wv_ranking = []
  max_validation_examples = 1000
  for i, line in enumerate(tqdm(validation_data)):
      if i == max_validation_examples:
          break
      q, *ex = line
      ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
      
      wv_ranking.append([r[0] for r in ranks][0]+1) 

  # среди всех дубликатов (r[0] for r in ranks) выбираем номер самого лучшего дубликата, 
  # т.е. стоящего на первом месте ([0]) и прибавляем к нему единицу, 
  # т.к. для подсчета метрик ранги должны начинаться с 1, а не с 0

  for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % \
          (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))
  print()

re



  0%|          | 0/3760 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.395 | Hits@   1: 0.395
DCG@   5: 0.397 | Hits@   5: 0.399
DCG@  10: 0.398 | Hits@  10: 0.403
DCG@ 100: 0.409 | Hits@ 100: 0.463
DCG@ 500: 0.441 | Hits@ 500: 0.720
DCG@1000: 0.470 | Hits@1000: 1.000

re_lem



  0%|          | 0/3760 [00:00<?, ?it/s]

...

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.383 | Hits@   1: 0.383
DCG@   5: 0.385 | Hits@   5: 0.387
DCG@  10: 0.385 | Hits@  10: 0.388
DCG@ 100: 0.396 | Hits@ 100: 0.448
DCG@ 500: 0.429 | Hits@ 500: 0.711
DCG@1000: 0.459 | Hits@1000: 1.000

W_lem



  0%|          | 0/3760 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.419 | Hits@   1: 0.419
DCG@   5: 0.420 | Hits@   5: 0.422
DCG@  10: 0.422 | Hits@  10: 0.427
DCG@ 100: 0.433 | Hits@ 100: 0.488
DCG@ 500: 0.465 | Hits@ 500: 0.743
DCG@1000: 0.492 | Hits@1000: 1.000



### Замечание:
Решить эту задачу с помощью обучения полноценной нейронной сети будет вам предложено, как часть задания в одной из домашних работ по теме "Диалоговые системы".

## Выводы:

* Какой принцип токенизации даёт качество лучше и почему?

Я попробовала два вида токенизации - через регулярные выражения и через WordPunctTokenizer. В первом случае в токены попадают только слова и цифры, во втором случае - еще различные знаки, что для тематики StackOverflow, по идее, должно быть полезно. Но это предположение не сработало, во всех случаях WordPunctTokenizer проигрывал регулярным выражениям.

* Помогает ли нормализация слов?

Для данной задачи нельзя сказать однозначно. Для предобученных эмбедингов нормализация слов приводила к понижению качества, а вот для эмбедингов, обученных нами - наоборот, к повышению.

* Какие эмбеддинги лучше справляются с задачей и почему?

Без лемматизации лучше справляются предобученные эмбединги, а с лемматизацией - лучше самостоятельно обученные эмбединги. Т.е. специфика корпуса текста для получения эмбедингов лучше всего проявляется при лемматизации - меньше мусора - слов не по теме, меньше словарь, четче видна связь между словами.

* Почему получилось плохое качество решения задачи?

Использование среднего по эмбедингам слов для эмбединга предложения - скорее всего достаточно примитивное решение, не учитывающее разнообразие слов.